#0 - Basic Settings

In [ ]:
#Permission to access some file from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Increasing the display capacity of columns and rows
import pandas as pd

pd.set_option('display.max_columns', 7000)
pd.set_option('display.max_rows',90000)

#1 - Generation of the target attribute (Deleteria11) in the database with 33 cancers

The target attribute will be calculated:
- Deleteria11



In [ ]:
#Reading Base_33TecidosAlphaFold_clean.csv database
import pandas as pd
base_33 = pd.read_csv("drive/My Drive/ProcessaNovaBase/TratamentosFinais_Pos_RINGeBetwennessClust_BaseAlphaFold/Base_33TecidosAlphaFold_clean.csv", low_memory = False, delimiter='\t')

In [ ]:
base_33.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201664 entries, 0 to 201663
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CHROM                         201664 non-null  int64  
 1   POS                           201664 non-null  int64  
 2   REF                           201664 non-null  object 
 3   ALT                           201664 non-null  object 
 4   Interpro_domain               201664 non-null  object 
 5   dbNSFP_DEOGEN2_pred           201664 non-null  object 
 6   dbNSFP_MetaSVM_pred           201664 non-null  object 
 7   dbNSFP_fathmmMKL_coding_pred  201664 non-null  object 
 8   dbNSFP_PrimateAI_pred         201664 non-null  object 
 9   dbNSFP_PROVEAN_pred           201664 non-null  object 
 10  dbNSFP_MCAP_pred              201664 non-null  object 
 11  dbNSFP_ClinPred_pred          201664 non-null  object 
 12  dbNSFP_BayesDel_addAF_pred    201664 non-nul

In [ ]:
#Identify duplicates records in the data
dupes=base_33.duplicated()
sum(dupes)

0

##1.1 Deleteria11

The Target attribute will be the **Deleteria11** attribute.

The **Deleteria11** attribute will be calculated as follows:

- Expression1: The **SIFT** attribute, or the **PolyPhen2_Dan** attribute   or the  **PROVEAN** attribute should have the value **D** or **P**.

- Expression2: The **dbNSFP_gnomAD_exomes_AF** attribute should be less than 10 raised to -4.

- Expression3:The **NdamageCalc** attribute should be >= 11

- Expression4: The **COMMON** attribute should be 0

The **Deleteria11** attribute  = Expression1 **and** Expression2 **and** Expression3 **and** Expression4

In [ ]:
#The or and and python statements require truth-values.
#For pandas these are considered ambiguous so you should use "bitwise" | (or) or & (and) operations:
#Extracted from https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o

SIFT = base_33["dbNSFP_SIFT_pred"].apply(lambda x:True if (x == "D" or x == "P") else False)
PolyPhen2 = base_33["PolyPhen2_Dam_pred"].apply(lambda x:True if (x == 1) else False)
PROVEAN = base_33['dbNSFP_PROVEAN_pred'].apply(lambda x:True if (x == "D" or x == "P") else False)
exp1 = SIFT | PolyPhen2 |PROVEAN

In [ ]:
exp2 = base_33['dbNSFP_gnomAD_exomes_AF'].apply(lambda x:True if (x < 0.0001) else False)

In [ ]:
exp3 = base_33['NdamageCalc'].apply(lambda x:True if (x >= 11) else False)

In [ ]:
exp4 = base_33['COMMON'].apply(lambda x:True if (x == 0) else False)

In [ ]:
base_33['Deleteria11'] = exp1 & exp2 & exp3 & exp4
base_33['Deleteria11'] = base_33['Deleteria11'].apply(lambda x:1 if(x == True) else 0)

In [ ]:
base_33.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201664 entries, 0 to 201663
Data columns (total 80 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CHROM                         201664 non-null  int64  
 1   POS                           201664 non-null  int64  
 2   REF                           201664 non-null  object 
 3   ALT                           201664 non-null  object 
 4   Interpro_domain               201664 non-null  object 
 5   dbNSFP_DEOGEN2_pred           201664 non-null  object 
 6   dbNSFP_MetaSVM_pred           201664 non-null  object 
 7   dbNSFP_fathmmMKL_coding_pred  201664 non-null  object 
 8   dbNSFP_PrimateAI_pred         201664 non-null  object 
 9   dbNSFP_PROVEAN_pred           201664 non-null  object 
 10  dbNSFP_MCAP_pred              201664 non-null  object 
 11  dbNSFP_ClinPred_pred          201664 non-null  object 
 12  dbNSFP_BayesDel_addAF_pred    201664 non-nul

In [ ]:
base_33[['dbNSFP_SIFT_pred','PolyPhen2_Dam_pred', 'dbNSFP_PROVEAN_pred', 'dbNSFP_gnomAD_exomes_AF', 'NdamageCalc', 'COMMON', 'Deleteria', 'Deleteria5', 'Deleteria10', 'Deleteria11']]

,dbNSFP_SIFT_pred,PolyPhen2_Dam_pred,dbNSFP_PROVEAN_pred,dbNSFP_gnomAD_exomes_AF,NdamageCalc,COMMON,Deleteria,Deleteria5,Deleteria10,Deleteria11
0,T,1,N,0.000279,3,0.0,0,0,0,0
1,D,1,D,0.000032,4,0.0,1,0,0,0
2,T,0,N,0.000016,5,0.0,0,0,0,0
3,T,1,N,0.000004,7,0.0,1,1,0,0
4,D,1,N,0.000068,10,0.0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...
201659,D,0,D,0.000044,3,0.0,0,0,0,0
201660,D,0,N,0.000016,8,0.0,1,1,0,0
201661,D,0,N,0.000000,14,0.0,1,1,1,1
201662,T,0,N,0.000004,4,0.0,0,0,0,0


In [ ]:
#Identify duplicates records in the data
dupes=base_33.duplicated()
sum(dupes)

0

In [ ]:
base_33['Deleteria10'].value_counts()

0    113985
1     87679
Name: Deleteria10, dtype: int64

In [ ]:
base_33['Deleteria11'].value_counts()

0    124457
1     77207
Name: Deleteria11, dtype: int64

##1.2 Generation of a file with the 33 cancer database with the *Deleteria11* attribute

In [ ]:
base_33.to_csv("drive/My Drive/ProcessaNovaBase/TratamentosFinais_Pos_RINGeBetwennessClust_BaseAlphaFold/Base_33TecidosAlphaFold_comDeleteria11.csv",sep='\t',index=False)